# Chapter9
In this chapter we look at:
* Syntax, code-idioms, and exhaustive search


# Syntax and Idioms
## Data Transport Container
The main way of storing model attributes and moving non-parallel models tests around on different CPUs

Class Definition is in /neuronunit/neuronunit/optimisation/data_transport_container.py

## Test Suite Dictionary
An extension of pythons dictionary class. It is capable of calling a method ```optimize``` on itself.

* This type also has an attribute, use_rheobase in scores.

* Since not all experimental cell test suites, use rheobase to judge model fitness (Allen SDK does not). It's simpler to decide if rheobase calculations will contribute to Neuronunit scores at the users entrypoint to optimization.

 /neuronunit/neuronunit/optimisation/optimization_management.py

## Class OptMan
* An Optimization Management Object
Located at
 /neuronunit/neuronunit/optimisation/optimization_management.py

This object contains the method ```run_ga```, ```round_trip_test``` and ```run_grid```.
The object contains logic for switching between elephant and allen based measurements of waveform and firing. This object also contains logic for rheobase guided searches, or many firing waveform searches.

Once an optimization has been described using the Optmization management object a secondary object is constructed.

The ```OptMan``` object also performs tasks like creating new genes to replace models that had unstable parameters

This Object constructs another object called  ```SciUnitOptimization``` that takes care of deap configuration. This file is departs from BluePyOpts base class, but it is similar to it.

The class sciunitoptimization largely calls methods in the files:
* ***/neuronunit/optimisation/algorithms.py***
* ***/neuronunit/optimisation/optimisations.py***

## These two files still largely follow BluePyOpt and DEAP conventions (expressed in DEAPS nsga2.py example file):

* ***/neuronunit/optimisation/algorithms.py***
* ***/neuronunit/optimisation/optimisations.py***

* With the differences: dask.bag.map is used to parallel map instead of scoop etc.

* nsga3 is used in the place of nsga2

* algorithms -> optimization_management.py ```def evaluate()``` 


In [ ]:
%%capture
from allensdk.ephys.extract_cell_features import extract_cell_features
import pickle
from neuronunit.optimisation.optimization_management import OptMan, TSD
from neuronunit.optimisation import model_parameters
from neuronunit.optimisation.model_parameters import MODEL_PARAMS
import pandas as pd
import os



In [ ]:

electro_path = str(os.getcwd())+'/../tests/russell_tests.p'

assert os.path.isfile(electro_path) == True
with open(electro_path,'rb') as f:
    (test_frame,obs_frame) = pickle.load(f)

In [ ]:
#filtered_tests = {key:val for key,val in test_frame.items()}
#spk_range = [spk-6,spk+6]
local_tests = test_frame['Hippocampus CA1 pyramidal cell']
local_tests = TSD(local_tests)
local_tests.use_rheobase_score = True
OM = OptMan(local_tests,backend=str('RAW'),boundary_dict=MODEL_PARAMS['RAW'],protocol={'elephant':True,'allen':False,'dm':False})#'tsr':spk_range})
grid_results = OM.run_simple_grid(5,free_params=['a','b','c'])
#res = OM.round_trip_test(local_tests,str('RAW'),MU=1,NGEN=1)#,stds = easy_standards)
[ (gr.dtc.scores,gr.dtc.attrs) for gr in grid_results ] 

In [ ]:
#grid_results[1].fitness

In [ ]:
#grid_results[1].dtc = OM.elephant_evaluation(grid_results[1].dtc)

In [ ]:
#grid_results[1].dtc.scores